## Feature Extraction

In order to feed text to a model we need to transform it to a numerical features, in this notebook we will discuss how to build a bag-of-words model from text to use it later for different applications.

---

### Bag of words

Count the occurrences of words in the corpus.

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
# CounterVectorizer (BOW)
texts = [ 'the red dog', 'cat eats dog', 'dog eats food',
         'red cat eats', 'the hot dog']
vectorizer = CountVectorizer()
vectorizer.fit(texts)

x = vectorizer.transform(texts)
columns = vectorizer.get_feature_names_out() # get words names 
pd.DataFrame(x.todense(), columns=columns, index=texts)

,cat,dog,eats,food,hot,red,the
the red dog,0,1,0,0,0,1,1
cat eats dog,1,1,1,0,0,0,0
dog eats food,0,1,1,1,0,0,0
red cat eats,1,0,1,0,0,1,0
the hot dog,0,1,0,0,1,0,1


-----

### Stop-words

Stop-words are words that are not significant to the topic in hand, for example `[am, is, are, in, at, ...]` can be considered stop-words in many applications as they don't add meaning.

In some other domains and problems you may have different kind of stop-words, for example if you are processing some chatbot data you may find `[can you please, would you please, can I, may I, ...]` such examples don't add meaning so stop-words can also be domain specific, and `TFIDF` can help you find these.

In [5]:
# CounterVectorizer (BOW) with stop words
texts = [ 'the red dog', 'cat eats dog', 'dog eats food',
         'red cat eats', 'the hot dog']
vectorizer = CountVectorizer(stop_words='english') # remove "the" 
vectorizer.fit(texts)
x = vectorizer.transform(texts)
columns = vectorizer.get_feature_names_out()
pd.DataFrame(x.todense(), columns=columns, index=texts) # sparse matrix to save memory in 1HotEnc 

,cat,dog,eats,food,hot,red
the red dog,0,1,0,0,0,1
cat eats dog,1,1,1,0,0,0
dog eats food,0,1,1,1,0,0
red cat eats,1,0,1,0,0,1
the hot dog,0,1,0,0,1,0


> Note that the word `the` was removed here

----

### N-Grams

N-Grams is a way we can use to count for the context in the text, the bigger n-gram range the bigger context you can capture but also more features to generate, so be careful not to break your memory.

In [6]:
# CounterVectorizer (BOW) with n_grams and stop words
texts = [ 'the red dog', 'cat eats dog', 'dog eats food',
         'red cat eats', 'the hot dog']
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))
vectorizer.fit(texts)
x = vectorizer.transform(texts)
columns = vectorizer.get_feature_names_out()
pd.DataFrame(x.todense(), columns=columns, index=texts)

,cat,cat eats,dog,dog eats,eats,eats dog,eats food,food,hot,hot dog,red,red cat,red dog
the red dog,0,0,1,0,0,0,0,0,0,0,1,0,1
cat eats dog,1,1,1,0,1,1,0,0,0,0,0,0,0
dog eats food,0,0,1,1,1,0,1,1,0,0,0,0,0
red cat eats,1,1,0,0,1,0,0,0,0,0,1,1,0
the hot dog,0,0,1,0,0,0,0,0,1,1,0,0,0


----

### TFIDF

Instead of just counting the frequency of each word, each word here is weighted using TF-IDF

$$W_{x, y} = tf_{x, y} \times log(\frac{N}{df_x})$$

In [7]:
# TF-IDF with stop words and n_grams
texts = [ 'the red dog', 'cat eats dog', 'dog eats food',
         'red cat eats', 'the hot dog']
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
vectorizer.fit(texts)
x = vectorizer.transform(texts)
columns = vectorizer.get_feature_names_out()
pd.DataFrame(x.todense(), columns=columns, index=texts) # sparse

,cat,cat eats,dog,dog eats,eats,eats dog,eats food,food,hot,hot dog,red,red cat,red dog
the red dog,0.000000,0.000000,0.401565,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.0000,0.575063,0.000000,0.712775
cat eats dog,0.460631,0.460631,0.321658,0.000000,0.382365,0.57094,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000
dog eats food,0.000000,0.000000,0.290314,0.515306,0.345106,0.00000,0.515306,0.515306,0.0000,0.0000,0.000000,0.000000,0.000000
red cat eats,0.437464,0.437464,0.000000,0.000000,0.363135,0.00000,0.000000,0.000000,0.0000,0.0000,0.437464,0.542226,0.000000
the hot dog,0.000000,0.000000,0.370086,0.000000,0.000000,0.00000,0.000000,0.000000,0.6569,0.6569,0.000000,0.000000,0.000000


----

We can already build some application using only these, let's try a very quick one

In [8]:
import numpy as np
import pandas as pd
from collections import Counter # data structure
import random
from termcolor import colored

# sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [9]:
# Load dataset
data = fetch_20newsgroups(subset='test', remove=['headers', 'footers', 'quotes'],
                         categories=['rec.autos', 'comp.windows.x', 
                                     'soc.religion.christian', 'rec.sport.baseball'])
X = data.data
y = [data.target_names[i] for i in data.target] # inverse transform num to text
print(f'DATA : {X[0]}')
print(f'LABEL: {y[0]}')

DATA : With all the recent problems the Indians have been having
with their pitching staff I have heard numerous names
thrown around about who could solve their problem.

One name I have not heard is Mike Soper (RP).  As far as
I know, Soper has had pretty good minor league stats.
Why not give the kid a chance?  Anyone know anything about
this guy?

-- 
LABEL: rec.sport.baseball


In [10]:
data.data

['With all the recent problems the Indians have been having\nwith their pitching staff I have heard numerous names\nthrown around about who could solve their problem.\n\nOne name I have not heard is Mike Soper (RP).  As far as\nI know, Soper has had pretty good minor league stats.\nWhy not give the kid a chance?  Anyone know anything about\nthis guy?\n\n-- ',
 '\n\t  The Catholic doctrine of predestination does not exclude free\n   will in any way.  Since God knows everything, He therefore knows\n   everything that is going to happen to us.  We have free will, and\n   are able to change what happens to us.  However, since God knows\n   everything, He knows all the choices we will make "in advance" (God\n   is not subject to time).  Too often arguments pit predestination\n   against free will.  We believe in both.\n\nThat last sentence of Steve\'s is an important one to remember.\n\nThere are certain things in the Catholic religion that cannot be\ncompletely comprehended by a human bein

In [15]:
data.target

array([2, 3, 2, ..., 1, 0, 0])

In [16]:
data.target_names

['comp.windows.x', 'rec.autos', 'rec.sport.baseball', 'soc.religion.christian']

In [17]:
Counter(y) # balanced

Counter({'rec.sport.baseball': 397,
         'soc.religion.christian': 398,
         'comp.windows.x': 395,
         'rec.autos': 396})

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Let's try to get the top-5 similar articles

In [19]:
vectorizer = CountVectorizer(stop_words='english', max_features=1000, max_df=0.7, min_df=0.01) #max & min %
vectorizer.fit(X_train) # vect for features X
X_train_v = vectorizer.transform(X_train)
X_test_v = vectorizer.transform(X_test)

In [20]:
X_train_v.shape, X_test_v.shape

((1268, 1000), (318, 1000))

* `Using Cosine Similairty`

In [21]:
for i in random.choices(range(0, len(X_test)), k=5):
    print(f"ID: {i}")
    print("True label:", colored(y_test[i], 'green'))
    distances = cosine_similarity(X_test_v[i], X_train_v).flatten() # or ravel or squeeze
    indices = np.argsort(distances)[::-1] # sort ascending and return index
    for _, j in enumerate(indices[:3]): # get the most 3
        print(f"{_} nearest label is {colored(y_train[j], 'green' if y_train[j]==y_test[i] else 'red')}",
             f"similarity: {colored(round(distances[j], 3), 'yellow')}")

ID: 253
True label: comp.windows.x
0 nearest label is comp.windows.x similarity: 0.318
1 nearest label is comp.windows.x similarity: 0.315
2 nearest label is comp.windows.x similarity: 0.279
ID: 231
True label: soc.religion.christian
0 nearest label is soc.religion.christian similarity: 0.35
1 nearest label is soc.religion.christian similarity: 0.334
2 nearest label is soc.religion.christian similarity: 0.298
ID: 246
True label: rec.autos
0 nearest label is soc.religion.christian similarity: 0.0
1 nearest label is rec.autos similarity: 0.0
2 nearest label is soc.religion.christian similarity: 0.0
ID: 184
True label: comp.windows.x
0 nearest label is comp.windows.x similarity: 0.463
1 nearest label is comp.windows.x similarity: 0.46
2 nearest label is comp.windows.x similarity: 0.361
ID: 139
True label: rec.autos
0 nearest label is soc.religion.christian similarity: 0.25
1 nearest label is rec.autos similarity: 0.236
2 nearest label is rec.sport.baseball similarity: 0.224


In [22]:
# List to append in it the predicted of test labels
y_pred_test = []

# Loop over the entire test dataset
for i in range(len(X_test)):
    # Get the true label for the current test instance
    true_label = y_test[i]   
    # Compute cosine similarity between the test instance and all training instances
    distances = cosine_similarity(X_test_v[i], X_train_v).flatten() 
    # Get the indices of the training instances sorted by similarity in descending order
    indices = np.argsort(distances)[::-1]
    # Get the labels of the three nearest neighbors
    nearest_labels = [y_train[j] for j in indices[:3]]
    # Determine the most common label among the three nearest neighbors
    y_pred_each = Counter(nearest_labels).most_common(1)[0][0]
    # Append to list
    y_pred_test.append(y_pred_each)

# Get Accuracy score
acc = accuracy_score(y_test, y_pred_test)
print(f'Acccuray Score using cosine simlarity is: {acc*100:.3f} %')

Acccuray Score using cosine simlarity is: 80.503 %


* `Using Euclidean Distance`

In [23]:
for i in random.choices(range(0, len(X_test)), k=5):
    print(f"ID: {i}")
    print("True label:", colored(y_test[i], 'green'))
    distances = euclidean_distances(X_test_v[i], X_train_v).flatten() 
    indices = np.argsort(distances)
    for _, j in enumerate(indices[:3]):
        print(f"{_} nearest label is {colored(y_train[j], 'green' if y_train[j]==y_test[i] else 'red')}",
             f"similarity: {colored(round(distances[j], 3), 'yellow')}")

ID: 35
True label: soc.religion.christian
0 nearest label is rec.autos similarity: 5.916
1 nearest label is rec.autos similarity: 5.916
2 nearest label is rec.autos similarity: 6.0
ID: 68
True label: rec.autos
0 nearest label is rec.sport.baseball similarity: 6.481
1 nearest label is rec.autos similarity: 6.481
2 nearest label is comp.windows.x similarity: 6.633
ID: 100
True label: rec.sport.baseball
0 nearest label is rec.sport.baseball similarity: 4.243
1 nearest label is rec.sport.baseball similarity: 4.472
2 nearest label is soc.religion.christian similarity: 4.472
ID: 282
True label: rec.sport.baseball
0 nearest label is rec.sport.baseball similarity: 5.657
1 nearest label is rec.sport.baseball similarity: 7.746
2 nearest label is rec.sport.baseball similarity: 7.81
ID: 153
True label: rec.sport.baseball
0 nearest label is rec.sport.baseball similarity: 6.633
1 nearest label is rec.autos similarity: 6.708
2 nearest label is rec.autos similarity: 6.782


In [24]:
# List to append in it the predicted of test labels
y_pred_test = []

# Loop over the entire test dataset
for i in range(len(X_test)):
    # Get the true label for the current test instance
    true_label = y_test[i]   
    # Compute euclidean_distances between the test instance and all training instances
    distances = euclidean_distances(X_test_v[i], X_train_v).flatten() 
    # Get the indices of the training instances sorted by distance in ascending order
    indices = np.argsort(distances)
    # Get the labels of the three nearest neighbors
    nearest_labels = [y_train[j] for j in indices[:3]]
    # Determine the most common label among the three nearest neighbors
    y_pred_each = Counter(nearest_labels).most_common(1)[0][0]
    # Append to list
    y_pred_test.append(y_pred_each)

# Get Accuracy score
acc = accuracy_score(y_test, y_pred_test)
print(f'Acccuray Score using Euclidean Distance is: {acc*100:.3f} %')

Acccuray Score using Euclidean Distance is: 53.774 %


* `Using Dot Product`

In [16]:
for i in random.choices(range(0, len(X_test)), k=5):
    print(f"ID: {i}")
    print("True label:", colored(y_test[i], 'green'))
    distances = (X_test_v[i] * X_train_v.T).toarray().flatten()  # dot product
    indices = np.argsort(distances)[::-1]
    for _, j in enumerate(indices[:3]):
        print(f"{_} nearest label is {colored(y_train[j], 'green' if y_train[j]==y_test[i] else 'red')}",
             f"similarity: {colored(round(distances[j], 3), 'yellow')}")

ID: 285
True label: comp.windows.x
0 nearest label is comp.windows.x similarity: 229
1 nearest label is comp.windows.x similarity: 146
2 nearest label is comp.windows.x similarity: 117
ID: 200
True label: rec.autos
0 nearest label is comp.windows.x similarity: 54
1 nearest label is comp.windows.x similarity: 24
2 nearest label is comp.windows.x similarity: 13
ID: 159
True label: rec.autos
0 nearest label is comp.windows.x similarity: 10
1 nearest label is soc.religion.christian similarity: 8
2 nearest label is rec.autos similarity: 5
ID: 175
True label: soc.religion.christian
0 nearest label is soc.religion.christian similarity: 408
1 nearest label is soc.religion.christian similarity: 294
2 nearest label is soc.religion.christian similarity: 210
ID: 42
True label: soc.religion.christian
0 nearest label is soc.religion.christian similarity: 94
1 nearest label is soc.religion.christian similarity: 88
2 nearest label is comp.windows.x similarity: 84


In [25]:
# List to append in it the predicted of test labels
y_pred_test = []

# Loop over the entire test dataset
for i in range(len(X_test)):
    # Get the true label for the current test instance
    true_label = y_test[i]   
    # Compute Dot product between the test instance and all training instances
    distances = (X_test_v[i] * X_train_v.T).toarray().flatten()  # dot product
    # Get the indices of the training instances sorted by similarity in descending order
    indices = np.argsort(distances)[::-1]
    # Get the labels of the three nearest neighbors
    nearest_labels = [y_train[j] for j in indices[:3]]
    # Determine the most common label among the three nearest neighbors
    y_pred_each = Counter(nearest_labels).most_common(1)[0][0]
    # Append to list
    y_pred_test.append(y_pred_each)

# Get Accuracy score
acc = accuracy_score(y_test, y_pred_test)
print(f'Acccuray Score using Euclidean Distance is: {acc*100:.3f} %')

Acccuray Score using Euclidean Distance is: 50.000 %


---